# Converting from Qiskit Runtime Programs

This tutorial will be a demonstation of converting your custom Qiskit Runtime Program into a Quantum Serverless `Program`.

If you were using Qiskit Runtime Programs before, your code probably looks similar to the following example:

```python
"""A sample runtime program that submits random circuits for user-specified iterations."""

import random

from qiskit import transpile
from qiskit.circuit.random import random_circuit


def prepare_circuits(backend):
    circuit = random_circuit(
        num_qubits=5, depth=4, measure=True, seed=random.randint(0, 1000)
    )
    return transpile(circuit, backend)


def main(backend, user_messenger, **kwargs):
    """Main entry point of the program.

    Args:
        backend: Backend to submit the circuits to.
        user_messenger: Used to communicate with the program consumer.
        kwargs: User inputs.
    """
    iterations = kwargs.pop("iterations", 5)
    for it in range(iterations):
        qc = prepare_circuits(backend)
        result = backend.run(qc).result()
        user_messenger.publish({"iteration": it, "counts": result.get_counts()})

    return "Hello, World!"
```


All Qiskit Runtime Programs have a `main` method which accepts `backend`, `user_messenger` and `**kwargs`. This method is not required for Quantum Serverless programs.

Quantum Serverless handles backends, logging, and input arguments a bit differently than Qiskit Runtime:

- `backend`. For Quantum Serverless programs you are not limited to single backend for a program. You can call any
  number of backends from single program. Since `Backend.run` is deprecated, we will be using Qiskit Primitives to do our calculation.
- `user_messenger` were used in Qiskit Runtime Programs to facilitate retrieving logs from the program. Quantum Serverless does not
  require passing such an object. Instead, all contents of `stdout` (e.g. print statements, logging messages) will be provided to the
  user via the Quantum Serverless job handler.
- `**kwargs` was a variable used to capture program inputs from the user. Users should now input their arguments to the `Program` constructor,
  and the arguments should be retrieved within the program using the `get_arguments` function from Quantum Serverless.
- To save the results of a program, the `save_result` function should be used. It accepts a python dictionary and can be accessed via the job handler.

Let's use the guidelines above to transform the above Qiskit Runtime Program into a Quantum Serverless Program.

```python
# migrated_program.py
"""A sample runtime program that submits random circuits for user-specified iterations."""

import random

from qiskit import transpile
from qiskit.circuit.random import random_circuit
from qiskit.primitives import Sampler

from quantum_serverless import get_arguments, save_result


def prepare_circuits():
    circuit = random_circuit(
        num_qubits=5, depth=4, measure=True, seed=random.randint(0, 1000)
    )
    return transpile(circuit)


arguments = get_arguments()
iterations = arguments.get("iterations", 5)

for it in range(iterations):
    qc = prepare_circuits()
    result = Sampler.run(qc).result()
    print({"iteration": it, "dists": result.quasi_dists})

save_result({"result": "Hello, World!"})
```

Let's save this code as `./src/migrated_program.py` and execute it using the `Program` class from the `quantum_serverless` package.

In [4]:
from quantum_serverless import Program

program = Program(
    title="migrated-program", entrypoint="migrated_program.py", working_dir="./src/"
)

In [5]:
from quantum_serverless import ServerlessProvider
import os

serverless = ServerlessProvider(
    username=os.environ.get("GATEWAY_USER", "user"),
    password=os.environ.get("GATEWAY_PASSWORD", "password123"),
    # token=os.environ.get("GATEWAY_TOKEN", "<TOKEN>"), # token can be used instead of user/password combination
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)
serverless

<ServerlessProvider: gateway-provider>

In [6]:
serverless.upload(program)

'migrated-program'

While Qiskit Runtime programs required users to upload their program and call it in two separate steps, the ``Program`` class allows users to send a job for remote execution in a single step.

In [7]:
job = serverless.run("migrated-program", arguments={"iterations": 3})
job

<Job | 901efd9c-7bb1-42e6-ad6a-14e781930f1c>

In [8]:
job.result()

{'result': 'Hello, World!'}

In [9]:
print(job.logs())

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
{'iteration': 0, 'dists': [{0: 0.0870879477472883, 1: 0.0924709553907342, 4: 0.0870879477472882, 5: 0.0924709553907341, 8: 0.0363341145102879, 9: 0.0341069823516897, 12: 0.0363341145102879, 13: 0.0341069823516897, 16: 0.0870879477472883, 17: 0.0924709553907342, 20: 0.0870879477472882, 21: 0.0924709553907341, 24: 0.0363341145102879, 25: 0.0341069823516897, 28: 0.0363341145102879, 29: 0.0341069823516897}]}
{'iteration': 1, 'dists': [{0: 0.1788234278657898, 4: 0.1788234278657898, 8: 0.0410003465204518, 12: 0.0410003465204518, 16: 0.22791926492318, 20: 0.22791926492318, 24: 0.0522569606905783, 28: 0.0522569606905783}]}
{'iteration': 2, 'dists': [{0: 0.1850255799964094, 1: 0.100389938487105, 4: 0.139108126852289, 5: 0.0754763546641966, 16: 0.1850255799964094, 17: 0.100389938487105, 20: 0.139108126852289, 21: 0.075476354